# Feature Engineering

### Approach
<b>Features To Build </b>
- Total Songs Listened To
- App Page Interactions
    - Thumbs Up
    - Thumbs Down
    - Add Friend
    - Add to playlist
- Help Page Interactions/Error Page Interactions
- User Device Brand
- User Browser


<b> Key Points </b>
- The Feature dataframe will be created based on the userId
- Transformations will need to maximize Pyspark capabilities

In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import IntegerType, StringType,DoubleType
from pyspark.sql import functions as F
from pyspark.sql.functions import col, from_unixtime, date_trunc, udf, lit,date_format


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.express as px

In [2]:
def create_spark_session(app_name="Sparkify", default_settings = True ,  total_physical_cores=16,driver_memory = 8,executor_memory = 8):
# Calculate available cores for Spark
    try:
        spark.shutdown()
    except  Exception as e:
        print(e)

    if default_settings == False:
        total_physical_cores = input(" Available Cores")
        driver_memory =  input(" Driver Memory Allowance")
        executor_memory = input("Executor Memory Allowance")

    available_cores_for_spark =int( total_physical_cores - 2)
    # Configure Spark session
    spark = (
        SparkSession.builder.appName(app_name)
        .config("spark.driver.memory", str(int(driver_memory)) + "g")
        .config("spark.executor.memory", str(int(executor_memory)) + "g")
        .config("spark.executor.cores", available_cores_for_spark)
        .getOrCreate()
    )

    return  spark

spark = create_spark_session()

local variable 'spark' referenced before assignment


24/01/30 23:25:32 WARN Utils: Your hostname, Jacobs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.156 instead (on interface en0)
24/01/30 23:25:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/30 23:25:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/30 23:25:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
path = "../data/lg_sparkify_event_data.json"
sdf = spark.read.json(path)
display = False


In [4]:

clean_sdf = sdf.dropna(subset="userId")
clean_sdf.createOrReplaceTempView("cleaned_user_log")
unique_users = clean_sdf[["userId"]].distinct()

clean_sdf = clean_sdf.withColumn("ts", (col("ts") / 1000).cast("timestamp"))

# Apply date_format function
clean_sdf = clean_sdf.withColumn("date", date_format(col("ts"), "yyyy-MM-dd"))

# clean_sdf = clean_scopy_df.withColumn("ts", from_unixtime(col("ts") / 1000))  # Assuming ts is in milliseconds
# clean_sdf = sdf.withColumn("date", date_format(col("ts"), "yyyy-MM-dd"))

In [5]:
def handle_missing_users(
    sdf: DataFrame, unique_users: DataFrame, fill_value=lit(0)
) -> DataFrame:
    """
    Handle missing users in a PySpark DataFrame.

    Parameters:
    - sdf (DataFrame): PySpark DataFrame representing user data. Should have columns 'userId' and 'featur_name'.
    - unique_users (DataFrame): PySpark DataFrame with unique user information.

    Returns:
    - DataFrame: Updated PySpark DataFrame with filled missing users.
    """

    sdf_user_count = sdf.count()

    unique_count = unique_users.count()

    if sdf_user_count != unique_count:
        print(f"Missing Values: {unique_count - sdf_user_count}")
        missing_users = unique_users.select("userId").subtract(sdf.select("userId"))
        # Since the sdf is only two we rename the column based on sdf's second column
        missing_users_sdf = missing_users.withColumn(sdf.columns[1], fill_value)
        filled_missing_users = sdf.union(missing_users_sdf)

        return filled_missing_users
    else:
        return sdf
    
def flag_rows(df, column_name, check_list, flagged_column_name):
    """
    Flag rows in a PySpark DataFrame based on whether the value in a specified column is in a given list.

    Parameters:
    - df (pyspark.sql.DataFrame): The PySpark DataFrame to be modified.
    - column_name (str): The name of the column to check for values.
    - check_list (list): The list of values to check against.
    - flagged_column_name (str): The name of the new column to be created for the flags.

    Returns:
    pyspark.sql.DataFrame: The modified PySpark DataFrame with the new flagged column.

    This function takes a PySpark DataFrame, a column name, a list of values, and a flagged column name. It then adds a new column to the DataFrame
    that contains a flag (1 or 0) based on whether the values in the specified column are present in the given list.
    """

    def check_list_udf(value):
        return 1 if value in check_list else 0

    check_list_udf = udf(check_list_udf, IntegerType())

    df = df.withColumn(flagged_column_name, check_list_udf(df[column_name]))

    return df

## Build Features

## Label

In [6]:
labels = (
    clean_sdf[["userId", "page"]]
    .filter(col("page").isin(["Cancellation Confirmation"]))
    .drop_duplicates(["userId"])
)


labels = labels.withColumn("label", lit(1))
labels = labels.drop("page")

labels = handle_missing_users(labels, unique_users)

features_sdf = labels

if display == True:
    #Plot Counts
    df = labels.toPandas().groupby(by="label").count().reset_index()
    sns.barplot(data = df, x = "label", y = "userId", hue = "label")
    df.groupby("label").describe()

Missing Values: 17275


### Song Counts

In [7]:
song_counts = (
    clean_sdf[["userId", "artist"]].dropna(subset="artist").groupBy("userId").count()
)

song_counts =song_counts.withColumnRenamed("count","song_counts")
song_counts = handle_missing_users(song_counts, unique_users)


if display == True:
    df= labels.join(song_counts, "userId", "left_outer").select("label","song_counts").toPandas()
    avg_df = df.groupby("label").agg({'song_counts': 'mean'}).rename(columns={'song_counts': 'avg_song_counts'}).round()
    sns.barplot(data = avg_df, x = "label", y = "avg_song_counts", hue = "label")
    df_label_1 = df[df["label"] == 1].describe().drop(columns="label")
    df_label_0 = df[df["label"] == 0].describe().drop(columns="label")
    merged_df = pd.merge(df_label_1, df_label_0, left_index=True, right_index=True, suffixes=('_1', '_0'))
    merged_df

Missing Values: 17


### Distinct Artist

In [8]:
distinct_artist = (
    clean_sdf.filter(clean_sdf["artist"].isNotNull())
    .groupBy("userId")
    .agg(F.countDistinct("artist").alias("distinct_artist"))
)

distinct_artist = handle_missing_users(distinct_artist, unique_users)
if display == True:
    df= labels.join(distinct_artist, "userId", "left_outer").select("label","distinct_artist").toPandas()
    avg_df = df.groupby("label").agg({'distinct_artist': 'mean'}).rename(columns={'distinct_artist': 'avg_distinct_artist'}).round()

    sns.barplot(data = avg_df, x = "label", y = "avg_distinct_artist", hue = "label")
    df_label_1 = df[df["label"] == 1].describe().drop(columns="label")
    df_label_0 = df[df["label"] == 0].describe().drop(columns="label")
    merged_df = pd.merge(df_label_1, df_label_0, left_index=True, right_index=True, suffixes=('_1', '_0'))
    merged_df

Missing Values: 17


### User Level

In [9]:
user_level = (
    clean_sdf[["userId", "level", "ts"]]
    .orderBy("ts", ascending=False)
    .dropDuplicates(subset=["userId"])
    .select("userId", "level")
)

level_flag_udf = udf(lambda x: 1 if x == "paid" else 0, IntegerType())
# one-hot encode
user_level = user_level.withColumn(
    "level_flag", level_flag_udf(user_level["level"])
).select("userId", "level_flag")

user_level = handle_missing_users(user_level, unique_users)

if display == True:
    df= labels.join(user_level, "userId", "left_outer").select("label","level_flag").toPandas()
    df= df.groupby("label").value_counts().reset_index()
    sns.barplot(data = df,x="level_flag",y="count",hue = "label")
    df_label_1 = df[df["label"] == 1].describe().drop(columns="label")
    df_label_0 = df[df["label"] == 0].describe().drop(columns="label")
    merged_df = pd.merge(df_label_1, df_label_0, left_index=True, right_index=True, suffixes=('_1', '_0'))
    merged_df

### Positive App Usage

In [10]:
positive_usage_list = ["Thumbs Up", "Thumbs Down", "Add Friend", "Add to playlist"]

positive_usage = (
    clean_sdf[["userId", "page"]]
    .filter(col("page").isin(positive_usage_list))
    .groupBy("userId")
    .count()
)

positive_usage = positive_usage.withColumnRenamed("count", "pos_interactions")

positive_usage = handle_missing_users(positive_usage, unique_users)

if display == True:
    df= labels.join(positive_usage, "userId", "left_outer").select("label","pos_interactions").toPandas()
    avg_df = df.groupby("label").agg({'pos_interactions': 'mean'}).round()
    sns.barplot(data = avg_df, x = "label", y = "pos_interactions", hue = "label")
    df_label_1 = df[df["label"] == 1].describe().drop(columns="label")
    df_label_0 = df[df["label"] == 0].describe().drop(columns="label")
    merged_df = pd.merge(df_label_1, df_label_0, left_index=True, right_index=True, suffixes=('_1', '_0'))
    merged_df

Missing Values: 318


### Negative Interactions

In [11]:
neg_interactions_list = ["Error", "Help"]

neg_interactions = (
    clean_sdf[["userId", "page"]]
    .filter(col("page").isin(neg_interactions_list))
    .groupBy("userId")
    .count()
)


#Display
neg_interactions = neg_interactions.withColumnRenamed("count", "neg_interactions")
neg_interactions = handle_missing_users(neg_interactions, unique_users)



if display == True:
    df= labels.join(neg_interactions, "userId", "left_outer").select("label","neg_interactions").toPandas()
    avg_df = df.groupby("label").agg({'neg_interactions': 'mean'}).round()
    sns.barplot(data = avg_df, x = "label", y = "neg_interactions", hue = "label")
    df_label_1 = df[df["label"] == 1].describe().drop(columns="label")
    df_label_0 = df[df["label"] == 0].describe().drop(columns="label")
    merged_df = pd.merge(df_label_1, df_label_0, left_index=True, right_index=True, suffixes=('_1', '_0'))
    merged_df



Missing Values: 3412


### Unique Locations

In [12]:
unique_locations = (
    clean_sdf.filter(clean_sdf["location"].isNotNull())
    .groupBy("userId")
    .agg(F.countDistinct("location").alias("unique_locations"))
)


unique_locations = handle_missing_users(unique_locations, unique_users)


if display == True:
    #Display
    df= labels.join(unique_locations, "userId", "left_outer").select("label","unique_locations").toPandas()
    avg_df = df.groupby("label").agg({'unique_locations': 'mean'}).round()
    sns.barplot(data = avg_df, x = "label", y = "unique_locations", hue = "label")
    df_label_1 = df[df["label"] == 1].describe().drop(columns="label")
    df_label_0 = df[df["label"] == 0].describe().drop(columns="label")
    merged_df = pd.merge(df_label_1, df_label_0, left_index=True, right_index=True, suffixes=('_1', '_0'))
    merged_df

Missing Values: 1


### Avg Daily Listens

In [13]:
# Calculate the average daily listens per user
average_daily_listens = (
    clean_sdf.dropna(subset="artist")
    .groupBy("userId", "date")
    .agg(F.count("artist").alias("daily_listens"))
    .groupBy("userId")
    .agg(F.avg("daily_listens").alias("avg_daily_listens"))
)

average_daily_listens = average_daily_listens.withColumn("avg_daily_listens", F.round("avg_daily_listens"))



if display == True:
    #Display
    df= labels.join(average_daily_listens, "userId", "left_outer").select("label","avg_daily_listens").toPandas()

    avg_df = df.groupby("label").agg({'avg_daily_listens': 'mean'}).round()
    sns.barplot(data = avg_df, x = "label", y = "avg_daily_listens", hue = "label")

    df_label_1 = df[df["label"] == 1].describe().drop(columns="label")
    df_label_0 = df[df["label"] == 0].describe().drop(columns="label")
    merged_df = pd.merge(df_label_1, df_label_0, left_index=True, right_index=True, suffixes=('_1', '_0'))
    merged_df

### Page Counts

In [14]:
page_filter = ["Cancel", "Cancellation Confirmation", "NextSong"]
filtered_sdf = clean_sdf.select("userId", "page").filter(~F.col("page").isin(page_filter))

# Grouping and Counting
page_count_sdf = (
filtered_sdf
    .groupBy("userId", "page")
    .agg(F.count("*").alias("count"))
)

page_count_sdf=page_count_sdf.groupBy("userId").pivot('page').agg(F.sum("count")).fillna(0)




if display == True:
    df= labels.join(page_count_sdf, "userId", "left_outer").toPandas()
    df = df.loc[:, df.columns != "userId"].groupby("label").mean()
    num_columns = len(df.columns)

    fig, axes = plt.subplots(nrows=1, ncols=num_columns, figsize=(15, 6), sharey=True)

    for i, col in enumerate(df.columns):
        df[[col]].T.plot(kind='bar', stacked=True, ax=axes[i], legend=False)
        axes[i].set_ylabel("frequency")

    plt.tight_layout()
    plt.legend(title='Columns', bbox_to_anchor=(1.05, 1), loc='upper left',  labels="label")
    plt.show

In [15]:
%pip install ua_parser

Note: you may need to restart the kernel to use updated packages.


## Device and Browsers

In [16]:
def ua_check(sdf):
    from ua_parser import user_agent_parser

    def device_check(ua):
        if ua:
            ua = ua.replace("\\", "")
            parsed_string = user_agent_parser.Parse(ua)
            return parsed_string["os"]["family"]
        else:
            return "Other"

    def browser_check(ua):
        if ua:
            ua = ua.replace("\\", "")
            parsed_string = user_agent_parser.Parse(ua)
            return parsed_string["user_agent"]["family"]
        else:
            return "Other"

    device_check_udf = udf(device_check, StringType())
    browse_brand_udf = udf(browser_check, StringType())

    sdf = sdf.withColumn("device", device_check_udf(sdf["userAgent"]))
    sdf = sdf.withColumn("browser", browse_brand_udf(sdf["userAgent"]))

    return sdf




In [17]:

cols_to_encode = ["device","browser"]

def one_hot_encode(sdf,cols_to_encode):
    conditions = {}
    categories = {}

    for col in cols_to_encode:
        cats = [row[0] for row in sdf.select(col).distinct().collect()]
        conditions[col] = {f"{col} == '{cat}'": idx for idx, cat in enumerate(cats)}
        categories[col] = {cat: idx for idx, cat in enumerate(cats)}

    for col, cats_dict in categories.items():
        for cat, idx in cats_dict.items():
            expression = F.when(sdf[col] == cat, 1).otherwise(0)
            sdf = sdf.withColumn(f"{col}_{cat}", expression)
        sdf = sdf.drop(col)

    return sdf

ua_sdf = one_hot_encode(ua_check(sdf).select("userId","device","browser"),cols_to_encode)


# Using the max() function retains whether a user at anytime used multiple device/browsers
columns_to_max = [col for col in ua_sdf.columns if col != 'userId']
# Loop allows feature to retain original name
ua_sdf = ua_sdf.groupby("userId").agg(*[F.max(col).alias(col) for col in columns_to_max])

if display == True:
    #Display
    df= labels.join(ua_sdf, "userId", "left_outer").toPandas()

    df = df.loc[:, df.columns != "userId"].groupby("label").sum()
    num_columns = len(df.columns)

    fig, axes = plt.subplots(nrows=1, ncols=num_columns, figsize=(15, 6), sharey=True)

    for i, col in enumerate(df.columns):
        df[[col]].T.plot(kind='bar', stacked=True, ax=axes[i], legend=False)
        axes[i].set_ylabel("frequency")

    plt.tight_layout()
    plt.legend(title='Columns', bbox_to_anchor=(1.05, 1), loc='upper left', labels="label")
    plt.show

## Time Since Registration

In [18]:
time_since_registration = sdf.groupBy('userId', 'ts', 'registration').count() \
    .withColumn('life_time', ((F.col('ts') - F.col('registration'))/ 86400000)) \
    .groupBy('userId').agg(F.round((F.max('life_time'))).alias('days_since_registration'))





if display == True:
    df= labels.join(time_since_registration, "userId", "left_outer").toPandas()

    df_label_1 = df[df["label"] == 1].describe().drop(columns="label")
    df_label_0 = df[df["label"] == 0].describe().drop(columns="label")
    merged_df = pd.merge(df_label_1, df_label_0, left_index=True, right_index=True, suffixes=('_1', '_0'))
    merged_df


## Avg. Day Between Sessions

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Assuming you have a SparkSession named 'spark' and a DataFrame named 'your_data'
# Replace 'your_data' with the actual DataFrame containing the columns 'userId', 'sessionId', and 'ts'

# Assuming 'ts' is in timestamp format, otherwise, you may need to convert it.

# Step 1: Calculate time differences
window_spec = Window.partitionBy("userId", "sessionId").orderBy("ts")

session_duration = sdf.withColumn("session_duration",(F.col("ts") - F.lag("ts").over(window_spec))/ 10000)

# Step 2: Aggregate to find the average time between sessions for each user
average_time_between_sessions = session_duration.groupBy("userId").agg(
     F.round(F.avg("session_duration")).alias("avg_time_between_sessions")
)


if display == True:
     # Show the result
     average_daily_time_between_sessions = handle_missing_users(average_time_between_sessions, unique_users)

     average_daily_time_between_sessions.show()

## Unique Sessions

In [20]:
unique_sessions = sdf.select("userId","sessionId").groupBy("userId").agg(
    F.countDistinct("*").alias("unique_sessions")
)

unique_sessions = handle_missing_users(unique_sessions,unique_users)

In [21]:
feature_sdfs= [
    song_counts,
    average_daily_listens,
    user_level,
    positive_usage,
    neg_interactions,
    # unique_locations,
    page_count_sdf,
    ua_sdf,
    distinct_artist,
    time_since_registration,
    unique_sessions,
]

processed_features = labels
from pyspark.sql.functions import log1p, col


def build_features(processed_sdf, feature_sdfs, use_log):
    for sdf in feature_sdfs:
        if use_log == True:
            distinct_count = sdf.select(sdf.columns[1]).distinct().count()
            if distinct_count > 2:
                sdf = sdf.withColumn(sdf.columns[1], F.log1p(col(sdf.columns[1])))
            
        processed_sdf = processed_sdf.join(sdf, "userId", "left_outer")

    return processed_sdf.dropDuplicates().dropna()



### Export Features For Model Testing

In [22]:
df1 = build_features(labels,feature_sdfs,use_log= True)
df1.toPandas().to_csv("./lg_log4_all_features.csv")

24/01/30 23:32:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [23]:
df2 = build_features(labels,feature_sdfs,use_log= False)
df2.toPandas().to_csv("./lg_noLog_all_features.csv")

# Analyzing Selected Features

In [ ]:
# df = pd.read_csv("../data/lg_log_all_features.csv/part-00000-711a9c3b-a5f1-4956-8a38-8a3ab92fde37-c000.csv")
# df.rename(columns={"count":"song_counts"},inplace = True)
# df = df.drop(columns=['Unnamed: 0', 'userId'])


df = log_processed_feature.toPandas()


In [ ]:

for col in df.columns:
    if df[col].nunique() > 2:
        data = df[[col]].value_counts().reset_index()
        sns.scatterplot(data=data,x= col,y = "count")# ,bins=round(data.shape[0]))
        plt.show()

## Feature Correalations

In [ ]:
df_corr = df.corr()

# df_corr = df_corr[df_corr >.8]

plt.figure(figsize=(8, 6))
sns.heatmap(df_corr, annot=False, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Extracted Feature Correalations")
plt.show()

##  Feature Distributions

In [ ]:
def display_distributions(df):
    sns.set(style="whitegrid")

    numerical_columns = [col for col in df.columns if df[col].nunique() > 2]
    num_numerical_columns = len(numerical_columns)
    
    # Determine the number of rows and columns for subplots
    num_rows = (num_numerical_columns - 1) // 3 + 1
    num_cols = min(3, num_numerical_columns)

    plt.figure(figsize=(15, 5 * num_rows))

    for i, column in enumerate(numerical_columns, start=1):
        plt.subplot(num_rows, num_cols, i)
        sns.histplot(df[column], kde=True)
        plt.title(f'Distribution of {column}')

    plt.tight_layout()
    plt.show()

In [ ]:

def log_and_clip(df, display =False):
    feat_list = [col for col in df.columns]
    try:
        feat_list.remove('userId')
    except:
        pass


    def soft_clip(x, threshold=3,tolerance=1e-10):
        mask = np.isclose(x, 0, atol=0)
        return np.where(mask, x, np.tanh(x / threshold) * threshold)
    

    for col in df[feat_list]:
        if df[col].nunique() :
            # Clip and log for original columns
            df[col + '_clipped'] = df[col].clip(lower=df[col].quantile(0.01), upper=df[col].quantile(0.99))
            df[col + '_log'] = np.log1p(df[col])
            # df[col + '_softClip'] = soft_clip(df[col])

    for col in df[feat_list].columns:
        if col.endswith('_log'):
            if df[col].nunique() > 2:
                df[col + '_logClip'] = df[col].clip(lower=df[col].quantile(0.01), upper=df[col].quantile(0.99))



    if display == True:
        for val in df.columns:
            feat_columns = [col for col in df.columns if col.startswith(val)]
            if len(feat_columns) > 1:
                display_distributions(df[feat_columns])

    return df


log_and_clip(df1.toPandas()[['song_counts']],True)
log_and_clip(df2.toPandas()[['song_counts']],True)

## Skewness

In [ ]:
def check_skew(df,display=True):
    skew_df =df.copy()
    skew ={}
    for col in df.columns:
        skew[col+ "_skew"] = df[col].skew()
    skew_df = pd.DataFrame(data=[skew.values()], columns=skew.keys()).transpose().sort_values(by = 0 , ascending=False)
    if display:
        skew_df.plot(kind= "bar")

    return skew_df


In [ ]:
check_skew(df)

In [ ]:
clip_log_df = log_and_clip(df,False) 
log_cols = [col for col in clip_log_df.columns if col.endswith("_log")]
clip_cols = [col for col in clip_log_df.columns if col.endswith("_clipped")]
clipLog_cols = [col for col in clip_log_df.columns if col.endswith("_logClip")]
softClip_cols = [col for col in clip_log_df.columns if col.endswith("_softClip")]

check_skew(df)
check_skew(clip_log_df[log_cols])
# check_skew(clip_log_df[clip_cols])
# check_skew(clip_log_df[clipLog_cols])
# check_skew(clip_log_df[softClip_cols])

# Determining Outliers

In [ ]:


def detect_outliers(df,display =True):
    def count_outliers(series, threshold=3):
        z_scores = np.abs((series - series.mean()) / series.std())
        return (z_scores > threshold).sum()


    non_binary_cols = [col for col in df.columns if df[col].nunique() >2]
    outliers_count = df[non_binary_cols].apply(count_outliers)

    print("Number of outliers for each feature:")
    outliers_df = pd.DataFrame(data=outliers_count)
    outliers_df["total samples"] = len(df)
    outliers_df = outliers_df.sort_values(by=0, ascending=False).rename(columns={0: "Outliers"})

    # Create a stacked bar plot
    outliers_df.plot(kind="bar", y = "Outliers",colormap="viridis")

    if display == True:
    
        plt.legend(title="Total Samples", loc="upper left")

        plt.xlabel("Features")
        plt.ylabel("Count of Outliers")
        plt.title("Stacked Bar Graph of Outliers by Feature")

        plt.show()

    return outliers_df

detect_outliers(df,False)
detect_outliers(clip_log_df[log_cols], False)